## Preprocessing notebook
Created by Nikolay Pavlychev \
email: nikolaypavlychev@ya.ru

In [1]:
# Standard python libraries
import os
import requests

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch


/home/nikolaypavlychev/anaconda3/envs/drugs_ner_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sales_train = pd.read_csv('./data/sales_train.csv',sep=',',dtype=object)
shops = pd.read_csv('./data/shops.csv',sep=',',dtype=object)
items = pd.read_csv('./data/items.csv',sep=',',dtype=object)
item_categories = pd.read_csv('./data/item_categories.csv',sep=',',dtype=object)

In [3]:
# Join tables
print(items['item_id'].unique().shape[0], items['item_id'].shape[0],
      item_categories['item_category_id'].unique().shape[0], item_categories['item_category_id'].shape[0])

22170 22170 84 84


In [4]:
items = items.merge(item_categories,on='item_category_id',how='inner')

sales_train.head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.0,1.0
5,10.01.2013,0,25,2564,349.0,1.0
6,02.01.2013,0,25,2565,549.0,1.0
7,04.01.2013,0,25,2572,239.0,1.0
8,11.01.2013,0,25,2572,299.0,1.0
9,03.01.2013,0,25,2573,299.0,3.0


In [5]:
sales = sales_train.merge(items[['item_id','item_category_id']],on='item_id',how='left')


In [6]:
sales.to_csv('./data/sales_history_dataset.csv', sep=';',index=False)

## Dataset statistics

In [7]:
sales['date'] =pd.to_datetime(sales['date'],format="%d.%m.%Y")

print('Sales period:')
sales['date'].min(), ' - ',sales['date'].max()

Sales period:


(Timestamp('2013-01-01 00:00:00'), ' - ', Timestamp('2015-10-31 00:00:00'))

In [8]:
sales['ID'] = sales['shop_id'].astype(str) + '_' + sales['item_id'].astype(str)
print(f"data shape: {sales.shape}")
print(f"number of time series in data: {sales['ID'].nunique()}")

data shape: (2935849, 8)
number of time series in data: 424124


In [9]:
print('time series dataset stats:')

sales_agg = sales.groupby(by='ID').agg(min_date_block_num=('date_block_num','min'),
                                       max_date_block_num=('date_block_num','max'),
                                       min_date=('date','min'),
                                       max_date=('date','max'),
                                       count_obs=('date','count'))
sales_agg.head(10)

time series dataset stats:


,min_date_block_num,max_date_block_num,min_date,max_date,count_obs
ID,,,,,
0_1000,0,1,2013-01-03,2013-02-27,9
0_10004,1,1,2013-02-09,2013-02-09,1
0_1001,0,0,2013-01-03,2013-01-12,2
0_10012,0,1,2013-01-22,2013-02-19,3
0_1002,0,0,2013-01-09,2013-01-28,2
0_1003,0,0,2013-01-09,2013-01-17,2
0_10033,1,1,2013-02-26,2013-02-26,1
0_10038,1,1,2013-02-28,2013-02-28,1
0_10039,0,1,2013-01-26,2013-02-24,3


In [10]:
sales_agg = sales_agg.reset_index()
sales_agg['sales_days'] = (sales_agg['max_date'] - sales_agg['min_date'])
sales_agg['sales_days'] = sales_agg['sales_days'].apply(lambda x: x.days)


In [11]:
print('sales min time period: ', sales_agg['sales_days'].min())
print('sales max time period: ', sales_agg['sales_days'].max())
print('------------------------------------------------')
print('sales min count: ', sales_agg['count_obs'].min())
print('sales max count: ', sales_agg['count_obs'].max())

sales min time period:  0
sales max time period:  1033
------------------------------------------------
sales min count:  1
sales max count:  867


In [12]:
print('sales mean count: ', sales_agg['count_obs'].mean())

sales mean count:  6.922147768105554
